# COGS515 - Homework 1

> Name Surname

> Student No

**Don't forget to enter your name and student no above**
23/11/2020

In [ ]:
from collections import namedtuple
import random, time

### TicTacToe Problem Class
Below is the problem and game state formulation we used for the TicTacToe problem. Check the python notebook we used in the tutorial to reviews the details of these classes (it's available in Teams and ODTUClass).

In [ ]:
GameState = namedtuple('GameState','to_move, utility, board, moves')

In [ ]:
class TicTacToe:
    def __init__(self):
        moves = [(0,0),(0,1),(0,2),
                 (1,0),(1,1),(1,2),
                 (2,0),(2,1),(2,2)]
        self.initial = GameState(to_move = 'X', utility = 0, board = {}, moves = moves)
        
    def actions(self, state):
        # Returns legal moves (i.e. empty spaces)
        return state.moves
    
    def result(self, state, move):
        if move not in state.moves:
            return state # Illegal move
        
        # Board next turn
        board_next_turn = state.board.copy()
        board_next_turn[move] = state.to_move
        
        # Available moves next turn
        moves_next_turn = list(state.moves)
        moves_next_turn.remove(move)
        
        
        # Player next turn
        if state.to_move == 'X':
            player_next_turn = 'O'
        else:
            player_next_turn = 'X'
        
        # Utility next turn
        utility_next_turn = self.compute_utility(board_next_turn, move, state.to_move)
        
        return GameState(to_move = player_next_turn, utility = utility_next_turn, 
                         board = board_next_turn, moves = moves_next_turn)
                
    
    

    
    def compute_utility(self, board, move, player):
        if self.three_in_row(board, move, player):
            if player == 'X':
                return 1
            else:
                return -1
        else:
            return 0
  
    def terminal_test(self, state):
        return state.utility != 0 or len(state.moves) == 0

        
    def three_in_row(self, board, move, player):

        for (dx, dy) in ((0,1),(1,0),(1,1),(1,-1)):

            n = 0
            x, y = move
            while board.get((x,y)) == player:
                n = n + 1
                x = x + dx
                y = y + dy

            x, y = move
            while board.get((x,y)) == player:
                n = n + 1
                x = x - dx
                y = y - dy

            n = n - 1

            if n >= 3:
                return True

        return False


    
    def utility(self, state, player):
        if player == 'X':
            return state.utility
        else:
            return -state.utility
        
        
    def display(self, state):
        board = state.board
        for x in range(3):
            for y in range(3):
                print(board.get((x,y),'.'), end=' ')
            print()
            
    def play_game(self, player1, player2):
        state = self.initial
        players = [player1,player2]
        while True:
            for player in players:
                print('-----------------------')
                cur_player = state.to_move
                print(cur_player,'plays')
                move = player(self,state)
                state = self.result(state,move)
                self.display(state)
                if self.terminal_test(state):
                    print()
                    print('###########################')
                    if(self.utility(state,player) == 0):
                        print('Draw')
                    else:
                        print(cur_player,'wins!!!')
                    print('###########################')

                    return self.utility(state,player)

### Player - Random Player and Minimax Player
Below are the functions that represents player agents that plays randomly and those that play with minimax algortihm. Examine the minimax carefully to understand how the algortihm works.

In [ ]:
def random_player(game, state):
    if game.actions(state):
        return random.choice(game.actions(state))
    else:
        return None

In [ ]:
def minmax_player(game, state):
    player = state.to_move
    
    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state,player)
        v = -100
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v
            
    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state,player)
        v = 100
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v
    
    best_score = -100
    best_action = None

    for a in game.actions(state):
        v = min_value(game.result(state,a))
        if v > best_score:
            best_score = v
            best_action = a
    return best_action
  

### A game
To play a game, we will create a TicTacToe object from the class we defined above. Below we create one minmax and one random player and we pass these players to `play_game(player1, player2)` method to make them play to each other.

In [ ]:
ttt = TicTacToe()
p1 = minmax_player
p2 = minmax_player
%time ttt.play_game(p1,p2)

#### Note:
Minimax is used for both players to measure the execution time of minimax algorithm.

## Question 1 - Alpha Beta Pruning

Your first task is to implement alpha beta pruning algortihm to the function below. A part of the function has already been written for you. You need to implement within `max_value` and `min_value` functions that v is return only when it is greater than the beta or less than the alpha. 

A minmax player playing against a random player.

In [ ]:
def alphabeta_player(game, state):
    player = state.to_move
    
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state,player)
        v = -100
        for a in game.actions(state):
            
            ########## Implement how v should be updated here
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if beta <= alpha:
                break
            #################################################
            
            alpha = max(alpha, v)
        return v
            
    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state,player)
        v = 100
        for a in game.actions(state):
            
            ########## Implement how v should be updated here
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if beta <= alpha:
                break
            #################################################
            
            beta = min(beta, v)
        return v
    
    best_score = -100
    beta = 100
    best_action = None
    
    for a in game.actions(state):
        v = min_value(game.result(state,a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action
  

After implementing alpha beta algorithm, you can make it play against a random player by running the code below.

In [ ]:
ttt = TicTacToe()
p1 = alphabeta_player
p2 = alphabeta_player
%time ttt.play_game(p1,p2)

### Note:
Alpha-Beta Pruning is used on both players to measure the execution time of Alpha-Beta Pruning algorithm. Note that the time taken is significantly decreased compared to the Minimax algorithm while having exactly same moves and result.

## Question 2 - Gomoku

Your second task is to formulate the problem for the game **Gomoku**. Below a Gomoku class has been defined but the code inside the class is exactly the same as tictactoe. You need to update the `moves`, `three_in_row` and `display` methods according to the available moves and winning conditions of the game respectively.

In [ ]:
class Gomoku:
    def __init__(self):
        
        ### Change available moves for the Gomoku game. Try using a loop or list comprehension.
        moves = []
        for i in range(15):
            for j in range(15):
                moves.append((i,j))
        self.initial = GameState(to_move = 'X', utility = 0, board = {}, moves = moves)
        
    def actions(self, state):
        # Returns legal moves (i.e. empty spaces)
        return state.moves
    
    def result(self, state, move):
        if move not in state.moves:
            return state # Illegal move
        
        # Board next turn
        board_next_turn = state.board.copy()
        board_next_turn[move] = state.to_move
        
        # Available moves next turn
        moves_next_turn = list(state.moves)
        moves_next_turn.remove(move)
        
        
        # Player next turn
        if state.to_move == 'X':
            player_next_turn = 'O'
        else:
            player_next_turn = 'X'
        
        # Utility next turn
        utility_next_turn = self.compute_utility(board_next_turn, move, state.to_move)
        
        return GameState(to_move = player_next_turn, utility = utility_next_turn, 
                         board = board_next_turn, moves = moves_next_turn)
                
    
    

    
    def compute_utility(self, board, move, player):
        if self.five_in_row(board, move, player):
            if player == 'X':
                return 1
            else:
                return -1
        else:
            return 0
  
    def terminal_test(self, state):
        return state.utility != 0 or len(state.moves) == 0

        
    def five_in_row(self, board, move, player):

        for (dx, dy) in ((0,1),(1,0),(1,1),(1,-1)):
            ### Change winning conditions of the game for Gomoku (it needs to check if there is 5 in row)

            n = 0
            x, y = move
            while board.get((x,y)) == player:
                n = n + 1
                x = x + dx
                y = y + dy

            x, y = move
            while board.get((x,y)) == player:
                n = n + 1
                x = x - dx
                y = y - dy

            n = n - 1
            
            if n >= 5:
                return True

        return False


    
    def utility(self, state, player):
        if player == 'X':
            return state.utility
        else:
            return -state.utility
        
        
    def display(self, state):
        ### Change the function to display the board. Gomoku has a 15X15 board rather than 5.
        board = state.board
        for x in range(15):
            for y in range(15):
                print(board.get((x,y),'.'), end=' ')
            print()
            
    def play_game(self, player1, player2):
        state = self.initial
        players = [player1,player2]
        while True:
            for player in players:
                print('-----------------------')
                cur_player = state.to_move
                print(cur_player,'plays')
                move = player(self,state)
                state = self.result(state,move)
                self.display(state)
                if self.terminal_test(state):
                    print()
                    print('###########################')
                    if(self.utility(state,player) == 0):
                        print('Draw')
                    else:
                        print(cur_player,'wins!!!')
                    print('###########################')

                    return self.utility(state,player)

After adapting the problem formulation for Gomoku, make two players play again each other. (If you could not implement alpha beta pruning use a minimax player instead of alpha beta player below.

In [ ]:
ggame = Gomoku()
p1 = alphabeta_player
p2 = random_player
%time ggame.play_game(p1,p2)

### Note:
The time complexity of Gomoku is too big to solve with Alpha-Beta Pruning. However, the code is still modified for Gomoku problem. The best case I could find was to expand the board as 4x4 with three in a row check. The execution time was 17.1 seconds in that case. If both players are chosen as random_player, it can be seen that the Gomoku problem is correctly modified.

## Question 3 - Connect 4

Your second task is to formulate the problem for the game **Connnect 4**. Below a Connect 4 class has been defined but the code inside the class is exactly the same as tictactoe. You need to update the `moves`, `three_in_row` and `display` methods according to the available moves and winning conditions of the game respectively. You must also  update `actions` function because in Connect 4, we can only play to the lowest empty row in each column.

In [ ]:
class Connect 4:
    def __init__(self):
        
        ### Change available moves for the Connect 4 game. Try using a loop or list comprehension.
        moves = [(0,0),(0,1),(0,2),
                 (1,0),(1,1),(1,2),
                 (2,0),(2,1),(2,2)]
        self.initial = GameState(to_move = 'X', utility = 0, board = {}, moves = moves)
        
    def actions(self, state):
        # Returns legal moves 
        
        ### Change the function that returns legal actions in each state for connect 4 game. 
        ### Remember that in Connect 4 you can only make move to the lowest empty row in each column

        return state.moves
    
    def result(self, state, move):
        if move not in state.moves:
            return state # Illegal move
        
        # Board next turn
        board_next_turn = state.board.copy()
        board_next_turn[move] = state.to_move
        
        # Available moves next turn
        moves_next_turn = list(state.moves)
        moves_next_turn.remove(move)
        
        
        # Player next turn
        if state.to_move == 'X':
            player_next_turn = 'O'
        else:
            player_next_turn = 'X'
        
        # Utility next turn
        utility_next_turn = self.compute_utility(board_next_turn, move, state.to_move)
        
        return GameState(to_move = player_next_turn, utility = utility_next_turn, 
                         board = board_next_turn, moves = moves_next_turn)
                
    
    

    
    def compute_utility(self, board, move, player):
        if self.three_in_row(board, move, player):
            if player == 'X':
                return 1
            else:
                return -1
        else:
            return 0
  
    def terminal_test(self, state):
        return state.utility != 0 or len(state.moves) == 0

        
    def three_in_row(self, board, move, player):

        for (dx, dy) in ((0,1),(1,0),(1,1),(1,-1)):
            ### Change winning conditions of the game for Connect 4 (it needs to check if there is 4 in row)

            n = 0
            x, y = move
            while board.get((x,y)) == player:
                n = n + 1
                x = x + dx
                y = y + dy

            x, y = move
            while board.get((x,y)) == player:
                n = n + 1
                x = x - dx
                y = y - dy

            n = n - 1
            
            if n >= 3:
                return True

        return False


    
    def utility(self, state, player):
        if player == 'X':
            return state.utility
        else:
            return -state.utility
        
        
    def display(self, state):
        ### Change the function to display the board. Connect 4 is played on a 6 X 7 board.
        board = state.board
        for x in range(3):
            for y in range(3):
                print(board.get((x,y),'.'), end=' ')
            print()
            
    def play_game(self, player1, player2):
        state = self.initial
        players = [player1,player2]
        while True:
            for player in players:
                print('-----------------------')
                cur_player = state.to_move
                print(cur_player,'plays')
                move = player(self,state)
                state = self.result(state,move)
                self.display(state)
                if self.terminal_test(state):
                    print()
                    print('###########################')
                    if(self.utility(state,player) == 0):
                        print('Draw')
                    else:
                        print(cur_player,'wins!!!')
                    print('###########################')

                    return self.utility(state,player)

After implementing Connect 4, make two players play again each other. (If you could not implement alpha beta pruning use a minimax player instead of alpha beta player below.

In [ ]:
c4game = ConnectFour()
p1 = alphabeta_player
p2 = random_player
c4game.play_game(p1,p2)